In [2]:
#####生成真正的train.csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
import csv

train_data = pd.read_csv('data/train.csv')
count=0
for i in train_data['id']:
    count+=1    
print(count) ###1225029：0-1225028 ##train_data.shape[0]
file_dir="data/train_img/"

for i in range(0,count):
    _id=train_data['id'][i]
    _landId=train_data['landmark_id'][i]
    
    file_path="data/train-true.csv"
    
    img_path=file_dir+_id+".jpg"
    
    if os.path.exists(img_path):
        if os.path.exists(file_path):
            da = [_id, _landId]
            # 写入数据
            csvFile = open(file_path, "a+")
            writer = csv.writer(csvFile)
            # 写入的内容都是以列表的形式传入函数
            writer.writerow(da)
            csvFile.close()
        else:
            fileHeader = ["id", "landmark_id"]
            # 假设我们要写入的是以下两行数据
            da = [_id, _landId]
            # 写入数据
            csvFile = open(file_path, "a+")
            writer = csv.writer(csvFile)
            # 写入的内容都是以列表的形式传入函数
            writer.writerow(fileHeader)
            writer.writerow(da)
            csvFile.close()

1225029


In [1]:
#####按类别建立文件夹，
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
import csv
import shutil

train_data = pd.read_csv('data/train-true.csv')

img_dir_src="data/train_img/"
img_dir_dst="data/train_img2/"

for i in range(0,train_data.shape[0]):
    _id=train_data['id'][i]
    _landId=train_data['landmark_id'][i]
    
    #print("%d"%(_landId))
    sub_img_dir=img_dir_dst+"%d"%(_landId)
    img_path=img_dir_src+_id+".jpg"
    
    if os.path.exists(sub_img_dir):
        shutil.copy(img_path,sub_img_dir)
    else:
        os.makedirs(sub_img_dir)
        shutil.copy(img_path,sub_img_dir)

In [1]:
###################使用flow_from_directory
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image

import tensorflow as tf 
from sklearn.model_selection import train_test_split #数据拆分
from keras.utils import np_utils #编码器
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

import keras.backend.tensorflow_backend as KTF  
KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu':0}))) 

from inception_v4 import create_inception_v4

batch_size=16
epochs=20
train_data = pd.read_csv('data/train-true.csv')
num_of_imgs=train_data.shape[0]

#model = create_inception_v4() ###14944classes
model = load_model('temp2.h5')
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

train_images_path="data/train_img2/"

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=[0.5, 2.0],
    horizontal_flip=True,
channel_shift_range=20.)

train_generator = train_datagen.flow_from_directory(
        train_images_path,  
        target_size=(299, 299),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=True,
        seed=30)  

#test_datagen = ImageDataGenerator(rescale=1./255)
# validation_generator = test_datagen.flow_from_directory(
#         validate_images_path,
#         target_size=(64, 64),
#         batch_size=batch_size)
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')

steps_per_epoch = int(np.ceil(num_of_imgs / float(batch_size)))

history = model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        verbose=1,
        workers=4)

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1253: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


Found 1174459 images belonging to 14944 classes.
Epoch 1/20
73404/73404 [==============================] - 97071s 1s/step - loss: 7.8935 - acc: 0.0410
Epoch 2/20
73404/73404 [==============================] - 74266s 1s/step - loss: 7.8605 - acc: 0.0412
Epoch 3/20
 4894/73404 [=>............................] - ETA: 18:24:05 - loss: 7.8467 - acc: 0.0405

KeyboardInterrupt: 

In [2]:
# #testing
# scores=model.evaluate(X_test,y_test,verbose=1)
# print('Test score:', scores[0])
# print('Test accuracy:', scores[1])

# #model.save('edge_model2_random.h5')
#model.save('temp3.h5')
#train_generator.class_indices

In [7]:
1174459/11

106769.0

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        verbose=1,
        workers=4)

Epoch 1/20
 1845/73404 [..............................] - ETA: 19:47:59 - loss: 7.8561 - acc: 0.0407

In [3]:
#####预测并生成submission.csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
import csv

test_data = pd.read_csv('data/test-true.csv')
    
print(test_data.shape[0]) ###1225029：0-1225028 ##train_data.shape[0]
file_dir="data/test_img/"

for i in range(0,test_data.shape[0]):
    _id=test_data['id'][i]
    #_landId
    
    file_path="data/submission.csv"
    
    img_path=file_dir+_id+".jpg"
    im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
    imResize = im.resize((299, 299))
    imageData = np.array(imResize)

    X = imageData.reshape(1,299,299,3).astype('float32')
    
    pred = model.predict(X)
    class_i = np.argmax(pred[0])
    prob="%0.2f"%(pred[0][class_i])
    
    _landId=''
    for (key, value) in train_generator.class_indices.items():
        if value==class_i:
            _landId=key
    
    res=_landId+' '+prob
    
    
    if os.path.exists(img_path):
        if os.path.exists(file_path):
            da = [_id,res]
            # 写入数据
            csvFile = open(file_path, "a+")
            writer = csv.writer(csvFile)
            # 写入的内容都是以列表的形式传入函数
            writer.writerow(da)
            csvFile.close()
        else:
            fileHeader = ["id", "landmark_id"]
            # 假设我们要写入的是以下两行数据
            da = [_id,res]
            # 写入数据
            csvFile = open(file_path, "a+")
            writer = csv.writer(csvFile)
            # 写入的内容都是以列表的形式传入函数
            writer.writerow(fileHeader)
            writer.writerow(da)
            csvFile.close()

108333


KeyboardInterrupt: 

In [50]:
import json
jsObj = json.dumps(train_generator.class_indices)  
  
fileObject = open('class_indices.json', 'w+')  
fileObject.write(jsObj)  
fileObject.close()  